<a href="https://colab.research.google.com/github/AllamElsheikh/Ai_Journey-/blob/main/Predict_the_winner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predicting the winner in npa games

## 1-load the data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('nba_games.csv')
df.head()

,Unnamed: 0,mp,mp.1,fg,fga,fg%,3p,3pa,3p%,ft,...,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won
0,0,240.0,240.0,39.0,81.0,0.481,6.0,20.0,0.300,14.0,...,22.8,29.0,178.0,111.0,DAL,95,1,2016,2015-12-09,True
1,1,240.0,240.0,36.0,100.0,0.360,7.0,31.0,0.226,16.0,...,50.0,32.6,152.0,111.0,ATL,98,0,2016,2015-12-09,False
2,2,240.0,240.0,37.0,85.0,0.435,8.0,19.0,0.421,17.0,...,20.0,30.9,148.0,116.0,SAS,107,1,2018,2017-10-18,False
3,3,240.0,240.0,41.0,89.0,0.461,8.0,21.0,0.381,17.0,...,28.6,30.9,138.0,118.0,MIN,99,0,2018,2017-10-18,True
4,4,240.0,240.0,27.0,86.0,0.314,6.0,26.0,0.231,15.0,...,16.8,30.9,157.0,90.0,MEM,92,1,2021,2021-04-30,False


## 2-Preparring the data for ml model

In [3]:
df.shape

(17772, 151)

we see the date colume but not sored , so let's sort the df depending on it .

In [4]:
df = df.sort_values('date')
df.head()

,Unnamed: 0,mp,mp.1,fg,fga,fg%,3p,3pa,3p%,ft,...,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won
16086,16086,240.0,240.0,35.0,83.0,0.422,6.0,18.0,0.333,19.0,...,69.4,43.7,206.0,104.0,GSW,111,1,2016,2015-10-27,False
16904,16904,240.0,240.0,38.0,94.0,0.404,9.0,29.0,0.310,10.0,...,53.2,34.6,162.0,104.0,CHI,97,1,2016,2015-10-27,False
16905,16905,240.0,240.0,37.0,87.0,0.425,7.0,19.0,0.368,16.0,...,30.4,29.0,138.0,105.0,CLE,95,0,2016,2015-10-27,True
16087,16087,240.0,240.0,41.0,96.0,0.427,9.0,30.0,0.300,20.0,...,37.5,38.9,201.0,120.0,NOP,95,0,2016,2015-10-27,True
1225,1225,240.0,240.0,37.0,82.0,0.451,8.0,27.0,0.296,12.0,...,33.3,23.6,132.0,104.0,DET,106,0,2016,2015-10-27,False


In [5]:
df = df.reset_index(drop=True)
df.head()

,Unnamed: 0,mp,mp.1,fg,fga,fg%,3p,3pa,3p%,ft,...,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won
0,16086,240.0,240.0,35.0,83.0,0.422,6.0,18.0,0.333,19.0,...,69.4,43.7,206.0,104.0,GSW,111,1,2016,2015-10-27,False
1,16904,240.0,240.0,38.0,94.0,0.404,9.0,29.0,0.310,10.0,...,53.2,34.6,162.0,104.0,CHI,97,1,2016,2015-10-27,False
2,16905,240.0,240.0,37.0,87.0,0.425,7.0,19.0,0.368,16.0,...,30.4,29.0,138.0,105.0,CLE,95,0,2016,2015-10-27,True
3,16087,240.0,240.0,41.0,96.0,0.427,9.0,30.0,0.300,20.0,...,37.5,38.9,201.0,120.0,NOP,95,0,2016,2015-10-27,True
4,1225,240.0,240.0,37.0,82.0,0.451,8.0,27.0,0.296,12.0,...,33.3,23.6,132.0,104.0,DET,106,0,2016,2015-10-27,False


In [6]:
del df['mp.1']
del df['mp_opp.1']
del df['index_opp']
del df['Unnamed: 0']

**creating the target columne**

In [7]:
def creat_target(df):
  df['target'] = df['won'].shift(-1)
  return df
df = df.groupby('team' , group_keys=False).apply(creat_target)
df.head()

,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,...,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won,target
0,240.0,35.0,83.0,0.422,6.0,18.0,0.333,19.0,27.0,0.704,...,43.7,206.0,104.0,GSW,111,1,2016,2015-10-27,False,False
1,240.0,38.0,94.0,0.404,9.0,29.0,0.310,10.0,17.0,0.588,...,34.6,162.0,104.0,CHI,97,1,2016,2015-10-27,False,True
2,240.0,37.0,87.0,0.425,7.0,19.0,0.368,16.0,23.0,0.696,...,29.0,138.0,105.0,CLE,95,0,2016,2015-10-27,True,True
3,240.0,41.0,96.0,0.427,9.0,30.0,0.300,20.0,22.0,0.909,...,38.9,201.0,120.0,NOP,95,0,2016,2015-10-27,True,True
4,240.0,37.0,82.0,0.451,8.0,27.0,0.296,12.0,15.0,0.800,...,23.6,132.0,104.0,DET,106,0,2016,2015-10-27,False,True


In [8]:
df['team'][:5]

,team
0,NOP
1,CLE
2,CHI
3,GSW
4,ATL


In [9]:
df[df['team'] == 'ATL']

,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,...,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won,target
4,240.0,37.0,82.0,0.451,8.0,27.0,0.296,12.0,15.0,0.800,...,23.6,132.0,104.0,DET,106,0,2016,2015-10-27,False,True
39,240.0,42.0,83.0,0.506,10.0,24.0,0.417,18.0,26.0,0.692,...,36.9,185.0,116.0,NYK,101,1,2016,2015-10-29,True,True
58,240.0,36.0,83.0,0.434,8.0,23.0,0.348,17.0,22.0,0.773,...,26.4,116.0,103.0,CHO,94,0,2016,2015-10-30,True,True
82,240.0,37.0,88.0,0.420,7.0,29.0,0.241,13.0,14.0,0.929,...,27.9,300.0,113.0,CHO,92,1,2016,2015-11-01,True,True
117,240.0,37.0,90.0,0.411,7.0,21.0,0.333,17.0,22.0,0.773,...,34.5,143.0,112.0,MIA,92,1,2016,2015-11-03,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17613,240.0,29.0,75.0,0.387,10.0,36.0,0.278,23.0,27.0,0.852,...,35.7,229.0,104.0,MIA,115,1,2022,2022-04-17,False,False
17621,240.0,41.0,87.0,0.471,12.0,40.0,0.300,11.0,14.0,0.786,...,35.2,153.0,108.0,MIA,115,1,2022,2022-04-19,False,True
17641,240.0,41.0,80.0,0.513,12.0,32.0,0.375,17.0,21.0,0.810,...,31.3,182.0,122.0,MIA,110,0,2022,2022-04-22,True,False
17658,240.0,30.0,75.0,0.400,15.0,42.0,0.357,11.0,20.0,0.550,...,32.3,300.0,111.0,MIA,110,0,2022,2022-04-24,False,False


we note the last column is null in each team data
- we fill it with number 2

In [10]:
df['target'][pd.isnull(df['target'])] = 2

In [11]:
df['target'] = df['target'].astype(int , errors='ignore')

In [12]:
df['target'].value_counts()

,count
target,
1,8872
0,8870
2,30


check nulls in the df

In [13]:
nans = pd.isnull(df).sum()

In [14]:
nans = nans[nans > 0]

In [15]:
my_column = df.columns[~df.columns.isin(nans.index)]
my_column

Index(['mp', 'fg', 'fga', 'fg%', '3p', '3pa', '3p%', 'ft', 'fta', 'ft%',
       ...
       'usg%_max_opp', 'ortg_max_opp', 'drtg_max_opp', 'team_opp', 'total_opp',
       'home_opp', 'season', 'date', 'won', 'target'],
      dtype='object', length=142)

In [16]:
df = df[my_column].copy()

In [17]:
del_cols = ['date' , 'season' , 'won' , 'target' , 'team' , 'team_opp' ]

In [18]:
my_column = df.columns[~df.columns.isin(del_cols)]


In [19]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df[my_column] = scaler.fit_transform(df[my_column])

In [20]:
df

,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,...,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won,target
0,0.0,0.363636,0.338235,0.366029,0.206897,0.212121,0.395487,0.418605,0.412698,0.654609,...,0.277279,0.554502,0.317647,GSW,0.451923,1.0,2016,2015-10-27,False,0
1,0.0,0.431818,0.500000,0.322967,0.310345,0.378788,0.368171,0.209302,0.253968,0.519253,...,0.160462,0.345972,0.317647,CHI,0.317308,1.0,2016,2015-10-27,False,1
2,0.0,0.409091,0.397059,0.373206,0.241379,0.227273,0.437055,0.348837,0.349206,0.645274,...,0.088575,0.232227,0.329412,CLE,0.298077,0.0,2016,2015-10-27,True,1
3,0.0,0.500000,0.529412,0.377990,0.310345,0.393939,0.356295,0.441860,0.333333,0.893816,...,0.215661,0.530806,0.505882,NOP,0.298077,0.0,2016,2015-10-27,True,1
4,0.0,0.409091,0.323529,0.435407,0.275862,0.348485,0.351544,0.255814,0.222222,0.766628,...,0.019255,0.203791,0.317647,DET,0.403846,0.0,2016,2015-10-27,False,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17767,0.0,0.340909,0.367647,0.313397,0.517241,0.515152,0.469121,0.302326,0.285714,0.693116,...,0.182285,0.208531,0.411765,GSW,0.413462,0.0,2022,2022-06-10,False,0
17768,0.0,0.500000,0.411765,0.471292,0.310345,0.545455,0.267221,0.279070,0.222222,0.844807,...,0.928113,1.000000,0.411765,BOS,0.288462,0.0,2022,2022-06-13,True,1
17769,0.0,0.272727,0.220588,0.344498,0.379310,0.424242,0.408551,0.465116,0.476190,0.623104,...,0.181001,0.630332,0.352941,GSW,0.384615,1.0,2022,2022-06-13,False,0
17770,0.0,0.340909,0.294118,0.373206,0.379310,0.363636,0.466746,0.232558,0.174603,0.903151,...,0.120668,0.459716,0.400000,GSW,0.375000,0.0,2022,2022-06-16,False,2


## 3-Model building

we have alot of columns we can't give it all for the model so we will use SequentialFeatureSelector for that

In [21]:
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import accuracy_score



In [24]:
model = LogisticRegression()
spliter = TimeSeriesSplit(n_splits = 2)
selector = SequentialFeatureSelector(model ,
                                     n_features_to_select = 20 ,
                                     direction = 'forward' ,
                                     cv = spliter)

In [25]:
selector.fit(df[my_column] , df['target'])

SequentialFeatureSelector(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=2, test_size=None),
                          estimator=LogisticRegression(),
                          n_features_to_select=20)

see which is the prefect predictores

In [43]:
predictores = list(my_column[selector.get_support()])
predictores

['mp',
 '3p%',
 'pf',
 'trb%',
 'usg%',
 'fta_max',
 '+/-_max',
 'ts%_max',
 'efg%_max',
 'usg%_max',
 'mp_opp',
 '3p_opp',
 'blk_opp',
 'trb%_opp',
 'blk%_opp',
 'usg%_opp',
 'ast_max_opp',
 'stl_max_opp',
 'blk_max_opp',
 'ast%_max_opp']

we will use the previouse seasones to predict the present season

In [48]:
def model_build(data, model, predictors, start=2, step=1):
    all_predictions = []

    seasons = sorted(data["season"].unique())

    for i in range(start, len(seasons), step):
        season = seasons[i]
        train = data[data["season"] < season]
        test = data[data["season"] == season]

        model.fit(train[predictors], train["target"])

        preds = model.predict(test[predictors])
        preds = pd.Series(preds, index=test.index)
        combined = pd.concat([test["target"], preds], axis=1)
        combined.columns = ["actual", "prediction"]

        all_predictions.append(combined)
    return pd.concat(all_predictions)

In [49]:
predictions = model_build(df, model, predictores)
predictions

,actual,prediction
5250,1,1
5251,1,0
5252,0,0
5253,1,0
5254,0,0
...,...,...
17767,0,0
17768,1,0
17769,0,1
17770,2,0


check the model accuracy

In [50]:
score = accuracy_score(predictions['actual'] , predictions['prediction'])
score

0.5477559495288292

## 4-model impor

In [51]:
df.groupby(["home"]).apply(lambda x: x[x["won"] == 1].shape[0] / x.shape[0])
df

,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,...,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won,target
0,0.0,0.363636,0.338235,0.366029,0.206897,0.212121,0.395487,0.418605,0.412698,0.654609,...,0.277279,0.554502,0.317647,GSW,0.451923,1.0,2016,2015-10-27,False,0
1,0.0,0.431818,0.500000,0.322967,0.310345,0.378788,0.368171,0.209302,0.253968,0.519253,...,0.160462,0.345972,0.317647,CHI,0.317308,1.0,2016,2015-10-27,False,1
2,0.0,0.409091,0.397059,0.373206,0.241379,0.227273,0.437055,0.348837,0.349206,0.645274,...,0.088575,0.232227,0.329412,CLE,0.298077,0.0,2016,2015-10-27,True,1
3,0.0,0.500000,0.529412,0.377990,0.310345,0.393939,0.356295,0.441860,0.333333,0.893816,...,0.215661,0.530806,0.505882,NOP,0.298077,0.0,2016,2015-10-27,True,1
4,0.0,0.409091,0.323529,0.435407,0.275862,0.348485,0.351544,0.255814,0.222222,0.766628,...,0.019255,0.203791,0.317647,DET,0.403846,0.0,2016,2015-10-27,False,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17767,0.0,0.340909,0.367647,0.313397,0.517241,0.515152,0.469121,0.302326,0.285714,0.693116,...,0.182285,0.208531,0.411765,GSW,0.413462,0.0,2022,2022-06-10,False,0
17768,0.0,0.500000,0.411765,0.471292,0.310345,0.545455,0.267221,0.279070,0.222222,0.844807,...,0.928113,1.000000,0.411765,BOS,0.288462,0.0,2022,2022-06-13,True,1
17769,0.0,0.272727,0.220588,0.344498,0.379310,0.424242,0.408551,0.465116,0.476190,0.623104,...,0.181001,0.630332,0.352941,GSW,0.384615,1.0,2022,2022-06-13,False,0
17770,0.0,0.340909,0.294118,0.373206,0.379310,0.363636,0.466746,0.232558,0.174603,0.903151,...,0.120668,0.459716,0.400000,GSW,0.375000,0.0,2022,2022-06-16,False,2


we will make a funciton that take 10 row of each team to preicit

In [54]:
df_rolling = df[list(my_column) + ["won", "team", "season"]]

def find_team_averages(team):
    rolling = team.select_dtypes(include='number').rolling(10).mean()
    return rolling

df_rolling = df_rolling.groupby(["team", "season"], group_keys=False).apply(find_team_averages)

In [55]:
df_rolling

,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,...,ast%_max_opp,stl%_max_opp,blk%_max_opp,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,total_opp,home_opp,season
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17767,0.0,0.381818,0.292647,0.428230,0.468966,0.477273,0.448100,0.434884,0.373016,0.764177,...,0.373394,0.0570,0.1113,0.471908,0.170603,0.431754,0.522353,0.348077,0.5,2022.0
17768,0.0,0.502273,0.364706,0.517703,0.455172,0.481818,0.440736,0.320930,0.282540,0.757993,...,0.546101,0.0716,0.1171,0.374109,0.321566,0.642654,0.564706,0.392308,0.4,2022.0
17769,0.0,0.354545,0.279412,0.404545,0.437931,0.465152,0.429572,0.434884,0.385714,0.736639,...,0.384060,0.0591,0.1113,0.483229,0.174711,0.438863,0.483529,0.350000,0.5,2022.0
17770,0.0,0.354545,0.294118,0.389952,0.434483,0.459091,0.431710,0.406977,0.357143,0.754142,...,0.393693,0.0572,0.1111,0.483229,0.172144,0.460190,0.472941,0.344231,0.5,2022.0


In [56]:
rolling_cols = [f"{col}_10" for col in df_rolling.columns]
df_rolling.columns = rolling_cols
df = pd.concat([df, df_rolling], axis=1)

In [57]:
df.dropna(inplace=True)

In [58]:
df

,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,...,ast%_max_opp_10,stl%_max_opp_10,blk%_max_opp_10,tov%_max_opp_10,usg%_max_opp_10,ortg_max_opp_10,drtg_max_opp_10,total_opp_10,home_opp_10,season_10
243,0.0,0.522727,0.382353,0.523923,0.344828,0.333333,0.457245,0.255814,0.238095,0.708285,...,0.311927,0.0628,0.0679,0.413522,0.124134,0.361611,0.449412,0.347115,0.4,2016.0
251,0.0,0.659091,0.426471,0.645933,0.620690,0.515152,0.562945,0.325581,0.238095,0.927655,...,0.412271,0.0613,0.0772,0.469497,0.219641,0.394787,0.531765,0.324038,0.5,2016.0
252,0.0,0.386364,0.382353,0.358852,0.206897,0.181818,0.445368,0.511628,0.412698,0.827305,...,0.245757,0.0625,0.1145,0.437841,0.138126,0.507109,0.360000,0.351923,0.6,2016.0
253,0.0,0.500000,0.382353,0.497608,0.344828,0.318182,0.475059,0.325581,0.349206,0.593932,...,0.283257,0.0646,0.0759,0.512159,0.133633,0.277251,0.388235,0.308654,0.4,2016.0
256,0.0,0.318182,0.132353,0.500000,0.275862,0.272727,0.432304,0.581395,0.444444,0.879813,...,0.307569,0.0741,0.0982,0.313312,0.179974,0.500000,0.471765,0.380769,0.5,2016.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17767,0.0,0.340909,0.367647,0.313397,0.517241,0.515152,0.469121,0.302326,0.285714,0.693116,...,0.373394,0.0570,0.1113,0.471908,0.170603,0.431754,0.522353,0.348077,0.5,2022.0
17768,0.0,0.500000,0.411765,0.471292,0.310345,0.545455,0.267221,0.279070,0.222222,0.844807,...,0.546101,0.0716,0.1171,0.374109,0.321566,0.642654,0.564706,0.392308,0.4,2022.0
17769,0.0,0.272727,0.220588,0.344498,0.379310,0.424242,0.408551,0.465116,0.476190,0.623104,...,0.384060,0.0591,0.1113,0.483229,0.174711,0.438863,0.483529,0.350000,0.5,2022.0
17770,0.0,0.340909,0.294118,0.373206,0.379310,0.363636,0.466746,0.232558,0.174603,0.903151,...,0.393693,0.0572,0.1111,0.483229,0.172144,0.460190,0.472941,0.344231,0.5,2022.0


### A-give the model some information about the next game

In [61]:
#creat the next game team opp
def next_team(team , col_name):
  next_col = team[col_name].shift(-1)
  return next_col

def add_col(df , col_name):
  return df.groupby('team' , group_keys=False).apply(lambda x:  next_team(x,col_name))

df['next_home'] = add_col(df , 'home')
df['next_team_opp'] = add_col(df , 'team_opp')
df['next_date'] = add_col(df , 'date')



In [62]:
df.head()

,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,...,tov%_max_opp_10,usg%_max_opp_10,ortg_max_opp_10,drtg_max_opp_10,total_opp_10,home_opp_10,season_10,next_home,next_team_opp,next_date
243,0.0,0.522727,0.382353,0.523923,0.344828,0.333333,0.457245,0.255814,0.238095,0.708285,...,0.413522,0.124134,0.361611,0.449412,0.347115,0.4,2016.0,0.0,BOS,2015-11-13
251,0.0,0.659091,0.426471,0.645933,0.620690,0.515152,0.562945,0.325581,0.238095,0.927655,...,0.469497,0.219641,0.394787,0.531765,0.324038,0.5,2016.0,1.0,BRK,2015-11-14
252,0.0,0.386364,0.382353,0.358852,0.206897,0.181818,0.445368,0.511628,0.412698,0.827305,...,0.437841,0.138126,0.507109,0.360000,0.351923,0.6,2016.0,0.0,MIN,2015-11-15
253,0.0,0.500000,0.382353,0.497608,0.344828,0.318182,0.475059,0.325581,0.349206,0.593932,...,0.512159,0.133633,0.277251,0.388235,0.308654,0.4,2016.0,0.0,CHI,2015-11-16
256,0.0,0.318182,0.132353,0.500000,0.275862,0.272727,0.432304,0.581395,0.444444,0.879813,...,0.313312,0.179974,0.500000,0.471765,0.380769,0.5,2016.0,0.0,CHO,2015-11-15


In [64]:
full = df.merge(df[rolling_cols + ["next_team_opp", "next_date", "team"]],
                left_on=["team", "next_date"],
                right_on=["next_team_opp", "next_date"])


In [65]:
full

,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,...,blk%_max_opp_10_y,tov%_max_opp_10_y,usg%_max_opp_10_y,ortg_max_opp_10_y,drtg_max_opp_10_y,total_opp_10_y,home_opp_10_y,season_10_y,next_team_opp_y,team_y
0,0.00,0.477273,0.500000,0.375598,0.379310,0.348485,0.483373,0.441860,0.396825,0.730455,...,0.1072,0.380294,0.273427,0.270616,0.478824,0.308654,0.6,2016.0,SAC,TOR
1,0.00,0.340909,0.250000,0.413876,0.310345,0.257576,0.509501,0.511628,0.412698,0.827305,...,0.1032,0.437212,0.124904,0.404739,0.408235,0.428846,0.2,2016.0,TOR,SAC
2,0.50,0.409091,0.455882,0.330144,0.482759,0.515152,0.437055,0.372093,0.412698,0.568261,...,0.0770,0.504403,0.153273,0.344076,0.384706,0.319231,0.7,2016.0,CLE,DET
3,0.25,0.545455,0.544118,0.416268,0.413793,0.454545,0.419240,0.186047,0.142857,0.883314,...,0.1113,0.467505,0.276508,0.352607,0.482353,0.316346,0.7,2016.0,GSW,TOR
4,0.00,0.340909,0.558824,0.186603,0.206897,0.469697,0.203088,0.139535,0.111111,0.854142,...,0.0766,0.413732,0.156739,0.470142,0.391765,0.436538,0.6,2016.0,DEN,NOP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15769,0.00,0.545455,0.426471,0.511962,0.448276,0.469697,0.440618,0.372093,0.365079,0.659277,...,0.1170,0.457128,0.235173,0.562085,0.552941,0.429808,0.4,2022.0,BOS,GSW
15770,0.00,0.477273,0.455882,0.409091,0.517241,0.590909,0.414489,0.255814,0.222222,0.766628,...,0.1113,0.471908,0.170603,0.431754,0.522353,0.348077,0.5,2022.0,GSW,BOS
15771,0.00,0.340909,0.367647,0.313397,0.517241,0.515152,0.469121,0.302326,0.285714,0.693116,...,0.1147,0.431761,0.242875,0.567773,0.575294,0.394231,0.4,2022.0,BOS,GSW
15772,0.00,0.500000,0.411765,0.471292,0.310345,0.545455,0.267221,0.279070,0.222222,0.844807,...,0.1113,0.483229,0.174711,0.438863,0.483529,0.350000,0.5,2022.0,GSW,BOS


In [68]:
removed_columns = list(full.columns[full.dtypes == "object"]) +del_cols
removed_columns


['team_x',
 'team_opp',
 'date',
 'next_team_opp_x',
 'next_date',
 'next_team_opp_y',
 'team_y',
 'date',
 'season',
 'won',
 'target',
 'team',
 'team_opp']

In [70]:
selected_columns = full.columns[~full.columns.isin(removed_columns)]
selector.fit(full[selected_columns], full["target"])

SequentialFeatureSelector(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=2, test_size=None),
                          estimator=LogisticRegression(),
                          n_features_to_select=20)

In [73]:
predictors = list(selected_columns[selector.get_support()])
predictors

['usg%',
 'usg%_opp',
 'usg%_10_x',
 '+/-_max_10_x',
 'orb%_opp_10_x',
 'blk%_opp_10_x',
 'usg%_opp_10_x',
 'ft%_max_opp_10_x',
 'trb_max_opp_10_x',
 'drtg_max_opp_10_x',
 'next_home',
 'stl_10_y',
 'trb%_10_y',
 'usg%_10_y',
 'fga_max_10_y',
 '+/-_max_10_y',
 'usg%_max_10_y',
 'ft%_opp_10_y',
 'blk_opp_10_y',
 'tov_opp_10_y']

In [77]:
predictions = model_build(full, model, predictors)
predictions

,actual,prediction
4677,1,1
4678,1,0
4679,1,1
4680,1,0
4681,1,1
...,...,...
15769,0,1
15770,1,1
15771,0,0
15772,1,0


In [78]:
accuracy_score(predictions["actual"], predictions["prediction"])


0.6323330629899973

### B- using deferant models

In [81]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier

In [82]:
dtc = DecisionTreeClassifier()
rfc = RandomForestClassifier()
gbc = GradientBoostingClassifier()

In [83]:
predictions = model_build(full, dtc, predictors)
accuracy_score(predictions["actual"], predictions["prediction"])


0.5536631522032982

In [84]:
predictions = model_build(full, rfc, predictors)
accuracy_score(predictions["actual"], predictions["prediction"])


0.6176444083986663

In [85]:
predictions = model_build(full, gbc, predictors)
accuracy_score(predictions["actual"], predictions["prediction"])


0.6243128773542399

**The best model is logictic regression**